# Fine tune Chinese bert with sentences

CS685 Spring 2022 <br />
Apr. 26, 2022<br />
Hongyu Tu <br />

In [3]:
import torch
import pickle
import datasets 
import pandas as pd
from utils import *
import matplotlib.pyplot as plt

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

### Load data

In [1]:
# name = '../data/danmu'

# df = pd.read_csv('{}.csv'.format(name))
# text_raw, label_raw = df['danmu'].to_list(), df['Category ID'].to_list()

# cat_dic, tmp = init_category_dic(), {}
# for i in list(cat_dic.keys()):
#     _, __, main_id = cat_dic[i]
#     tmp[i] = main_id
    
# t_l = list(set(tmp.values()))

# text, label = [], []
# for i in range(len(text_raw)):
#     if label_raw[i] in tmp.keys() \
#         and 'str' in str(type(text_raw[i])) \
#         and len(text_raw[i]) < 25:
#         text.append(text_raw[i])
#         l = t_l.index(tmp[label_raw[i]])
#         label.append(l)
        

# split_idx = int(len(label) * 0.9)
# dataset = datasets.DatasetDict({"train": datasets.Dataset.from_dict({"text": text[:split_idx], "label": label[:split_idx]}), \
#                                 "test": datasets.Dataset.from_dict({"text": text[split_idx:], "label": label[split_idx:]})})

# with open('danmu_chn_bert_full.pkl', 'wb') as f:
#     pickle.dump(dataset, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('danmu_chn_bert_full.pkl', 'rb') as f:
    dataset = pickle.load(f)

### Model tuning

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/304 [00:00<?, ?ba/s]

  0%|          | 0/34 [00:00<?, ?ba/s]

In [9]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-chinese", num_labels=len(t_l))

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [12]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [13]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [17]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", num_train_epochs = 15)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [19]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\tomtu\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000
  Num Epochs = 15
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1875


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.379289,0.287000
2,No log,2.799761,0.216000
3,No log,3.022844,0.215000


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


KeyboardInterrupt: 